In [1]:
'''
Adapated from: https://github.com/CVxTz/kinship_prediction/blob/master/code/vgg_face.py
to parse the images and data
This is NOT used to make the model!
Additionally, this is run on a Windows OS. Some code may not be need if it is on a UNIX or LINUX OS.
'''
import pandas as pd
from glob import glob #for finding files recursively
from collections import defaultdict

# PyTorch
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torchvision.transforms as transforms
# import torchvision.utils
# import torchvision.datasets as dset

# from torch import optim
# from torch.utils.data import DataLoader,Dataset
# from torchvision.models import *
# from torchvision.datasets import ImageFolder
# from torch.autograd import Variable
# from PIL import Image

In [2]:
!unzip '/content/train.zip' -d '/content/train'

Streaming output truncated to the last 5000 lines.
  inflating: /content/train/F0601/MID2/P12074_face1.jpg  
  inflating: /content/train/F0601/MID2/P11969_face4.jpg  
  inflating: /content/train/F0601/MID2/P11883_face1.jpg  
  inflating: /content/train/F0601/MID2/P11900_face1.jpg  
  inflating: /content/train/F0601/MID2/P06303_face1.jpg  
  inflating: /content/train/F0601/MID2/P11885_face1.jpg  
  inflating: /content/train/F0601/MID2/P11875_face1.jpg  
  inflating: /content/train/F0601/MID2/P06300_face7.jpg  
  inflating: /content/train/F0601/MID2/P11878_face1.jpg  
  inflating: /content/train/F0601/MID2/P11898_face1.jpg  
  inflating: /content/train/F0601/MID2/P06394_face4.jpg  
  inflating: /content/train/F0601/MID2/P11879_face1.jpg  
  inflating: /content/train/F0601/MID2/P11877_face1.jpg  
  inflating: /content/train/F0601/MID2/P06344_face2.jpg  
  inflating: /content/train/F0601/MID2/P06279_face1.jpg  
  inflating: /content/train/F0601/MID2/P06288_face1.jpg  
  inflating: /content

In [19]:
relationshipsCSV = "./train_relationships.csv"
train_images_folder = "./train/"
#what is the difference between train and train-face?


'\nIdea of hypertuning:\nusing different validation sets for each iteration\nmaybe instead of using only only family as the validation, we can use a group\nor change the validation set for each iteration of training the model\n'

In [40]:
available_images = glob(train_images_folder + "*/*/*.jpg")
'''
Only for windows pc:
'''
for a in range(0, len(availble_images)):
    available_images[a] = available_images[a].replace("\\", "/")

all_ppl = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in available_images] #all the people

In [68]:
#creating the training set
personPathFile = defaultdict(list)
for x in available_images:
    personPathFile[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

#read from the csv to create a list of tuples
relationships = pd.read_csv(relationshipsCSV)
relationship_pairs = [(row['p1'], row['p2']) for index, row in relationships.iterrows()] # Create a list of tuples
relationship_pairs = [x for x in relationship_pairs if x[0] in all_ppl and x[1] in all_ppl] #data cleaning

#converting into dictionary of person is related to this list of people
relationshipDict = defaultdict(list)
for item in relationship_pairs:
  relationshipDict[item[0]].append(item[1]) #you do not need to consider the item[1] because the CSV is already pretty clean



#create more data by finding the 'negative pair'
notRelationshipDict = defaultdict(list)
set_all_ppl = set(all_ppl)
#for each key(person) in the relationship dictionary, find the non-relations
for k,v in relationshipDict.items():
    notRelationshipDict[k]=list(set_all_ppl-set(v))
    

    
# convert both dictionaries into a list of tuples with 1, 0 as labels
# for each of the image, take the first image path stored in the dictionary made earlier
# then take each of the path file in the item
trainData = []
for k,v in relationshipDict.items():
    for relation in v:
        for item in personPathFile[k]:
                trainData.append((item, personPathFile[relation][0],1))
    
print("Current Length of positive relationships: ", len(trainData))
for k,v in notRelationshipDict.items():
    for i in range(0,len(v)//170): #random number 
        trainData.append((personPathFile[k][0],personPathFile[relation][0],0))

print("Total Length of all relationships: ", len(trainData))        
# Storing the list of tuples in a text file
with open('Cleaned_Data.txt', 'w') as file:
    for item in trainData:
        file.write(f"{item[0]}, {item[1]}, {item[2]}\n")
        
        
# trainData1 = []
# with open('Cleaned_Data.txt', 'r') as file:
#     for l in file:
#         eachTuple = l.strip().split(', ')
#         trainData1.append(((eachTuple[0]), eachTuple[1], int(eachTuple[2])))

Current Length of positive relationships:  21283
Total Length of all relationships:  42577


3416100

In [8]:
'''
Adapted from: https://github.com/ale-mauro/Kinship-Recognition/blob/main/KinshipRecognition_Naive_DenseNet161_ResNet.ipynb
Obtaining the training and validation set to use the following model:
https://keras.io/examples/vision/siamese_network/

The idea is to split to obtain a "triple" set:
1. an anchor
2. positive
3. negative
'''

'\nAdapted from: https://github.com/ale-mauro/Kinship-Recognition/blob/main/KinshipRecognition_Naive_DenseNet161_ResNet.ipynb\nObtaining the training and validation set to use the following model:\nhttps://keras.io/examples/vision/siamese_network/\n\nThe idea is to split to obtain a "triple" set:\n1. an anchor\n2. positive\n3. negative\n'

In [9]:
#use the pytorch built in Dataset function
class trainingDataset(Dataset):
    def __init__(self, _ImagesFolder, _data, _transform=None):
        self.ImagesFolder = _ImagesFolder
        self.data = _data #choose either train or val dataset to use
        self.transform = _transform


    def __getitem__(self,index):
        
        first_img_path = "train/"+self.relationships[index][0]+"/.jpg"
        second_img_path = "train/"+self.relationships[index][1]+"/.jpg"
        img1Opened = Image.open(first_img_path)
        img2Opened = Image.open(second_img_path)
        return img1Opened, img2Opened , self.relationships[index][2]

    def __len__(self):
        return len(self.data)#essential for choose the num of data in one epoch


In [69]:
from sklearn.model_selection import train_test_split
# ImagesFolder = dset.ImageFolder(root='train/')
BATCH_SIZE=64
IMG_SIZE=100
NUM_WORKERS = 8


trainSet, valSet = train_test_split(trainData, test_size=0.1, random_state=42)


#Training set and training loader
trainset = trainingDataset(imageFolderDataset=ImagesFolder,
                                        relationships=trainSet,
                                        transform=transforms.Compose([transforms.Resize((IMG_SIZE,IMG_SIZE)),
                                                                      transforms.ToTensor()
                                                                      ]))

valset = trainingDataset(imageFolderDataset=ImagesFolder,
                                        relationships=valSet,
                                        transform=transforms.Compose([transforms.Resize((IMG_SIZE,IMG_SIZE)),
                                                                      transforms.ToTensor()
                                                                      ]))



trainloader = DataLoader(trainset,
                        shuffle=True,
                        num_workers=NUM_WORKERS,
                        batch_size=BATCH_SIZE)


valloader = DataLoader(valset,
                        shuffle=True,
                        num_workers=NUM_WORKERS,
                        batch_size=BATCH_SIZE)


NameError: name 'trainingDataset' is not defined

'\nMore models:\nhttps://viso.ai/computer-vision/deepface/ \n'